In [283]:
x_train = []
y_train = []
with open('isolet1+2+3+4.data/isolet1+2+3+4.data') as f:
    lines = f.readlines()
    for line in lines:
        splitted = line.split(', ')
        label = float(splitted[-1])
        features = [float(x) for x in splitted[:-1]]
        x_train.append(features)
        y_train.append(label)


In [284]:
x_test = []
y_test = []
with open('isolet5.data/isolet5.data') as f:
    lines = f.readlines()
    for line in lines:
        splitted = line.split(', ')
        label = float(splitted[-1])
        features = [float(x) for x in splitted[:-1]]
        x_test.append(features)
        y_test.append(label)

In [285]:
len(x_train)

6238

In [358]:
import random
import numpy as np

class hd_classifier():
 
        
    
    def __init__(self, dim=5000):

        self.x_train = np.asarray(x_train)
        self.y_train = np.asarray(y_train)

        self.max_x_train = 1
        self.min_x_train = -1
        self.feature_len = len(self.x_train[0])
        self.label_len = 26
        self.dim = dim
        self.gamma = 0.5
        self.level_hv_min = self.random_hv_generator()
        self.level_hv_max = self.random_hv_generator()
        
        # initializing middle level-hypervectors
        self.level_hvs = [None for i in range(9)]
        for i in range(len(self.level_hvs)):
            self.level_hvs[i] = [0 for i in range(self.dim)]
            ratio = (i+1)/10
            for j in range(self.dim):
                if random.uniform(0, 1) < ratio:
                    self.level_hvs[i][j] = self.level_hv_max[j]
                else:
                    self.level_hvs[i][j] = self.level_hv_min[j]

    
        # generate identification vectors for association-based encoding
        self.id_assoc = [None for i in range(self.feature_len)]
        for i in range(len(self.id_assoc)):
            self.id_assoc[i] = self.random_hv_generator()
            
        # calc gaussian kernel matrix
        mu = 0
        sigma = 1
        self.kernel = np.asarray([[np.random.normal(mu, sigma, 1)[0] for i in range(self.feature_len)] for j in range(self.dim)])

        # bundled classifier hv for every label in perm-based encoding
        self.classes_perm_encodings = np.asarray([[0 for i in range(self.dim)] for j in range(self.label_len)])

        # bundled classifier hv for every label in assoc-based encoding
        self.classes_assoc_encodings = np.asarray([[0 for i in range(self.dim)] for j in range(self.label_len)])

        # bundled classifier hv for every label in kernel-based encoding
        self.classes_kernel_encodings = np.asarray([[0 for i in range(self.dim)] for j in range(self.label_len)])
    

    def random_hv_generator(self):
        hv = np.asarray([1 for i in range(self.dim)])
        for i in range(self.dim):
            if random.randint(0, 1) == 0:
                hv[i] = -1
        return hv

    def perm_hv(self, hv, perm_num):
        return np.roll(hv, perm_num)
    
    def assign_level_hv(self, e):
        result = None
        ratio = ((e+1)/2)*10
        
        if 0 <= ratio < 0.5:
            result = self.level_hv_min
        elif 0.5 <= ratio < 1.5:
            result = self.level_hvs[0]
        elif 1.5 <= ratio < 2.5:
            result = self.level_hvs[1]
        elif 2.5 <= ratio < 3.5:
            result = self.level_hvs[2]
        elif 3.5 <= ratio < 4.5:
            result = self.level_hvs[3]
        elif 4.5 <= ratio < 5.5:
            result = self.level_hvs[4]
        elif 5.5 <= ratio < 6.5:
            result = self.level_hvs[5]
        elif 6.5 <= ratio < 7.5:
            result = self.level_hvs[6]
        elif 7.5 <= ratio < 8.5:
            result = self.level_hvs[7]
        elif 8.5 <= ratio < 9.5:
            result = self.level_hvs[8]
        else:
            result = self.level_hv_max
        return result

    def calc_perm_enc(self, sample):
        ratio = 0
        encoded_sample = np.asarray([0 for i in range(self.dim)])
        for i, e in np.ndenumerate(sample):
            feature_hv = self.assign_level_hv(e)
            feature_hv = self.perm_hv(feature_hv, i[0])
            encoded_sample = np.add(encoded_sample, feature_hv)

        return encoded_sample
    
    
    def FV_perm_classifier(self):
        # encoding each feature and then bundling their permutations
        zipped_x_y_train = np.column_stack((self.x_train, self.y_train))

        for e in zipped_x_y_train:
            label = int(e[self.feature_len])
            sample = np.asarray(e[:self.feature_len])
            idx = label - 1
            self.classes_perm_encodings[idx] = np.add(self.classes_perm_encodings[idx], self.calc_perm_enc(sample))
        return
    
    def cosine(self, encoded_sample, classes):
        pred_idx = 0
        cosine_val = 0
        for i in range(self.label_len):
            sim_val = np.dot(encoded_sample, classes[i])
            if sim_val > cosine_val:
                pred_idx = i
                cosine_val = sim_val

        return (pred_idx + 1)
    
    
    def predict_perm(self, sample):
        perm_encoded_sample = self.calc_perm_enc(sample)
        predicted_label = self.cosine(perm_encoded_sample, self.classes_perm_encodings)
        return predicted_label
    
    
    def FV_adaptive_perm_classifier(self):
        # encoding each feature and then bundling their permutations
        zipped_x_y_train = np.column_stack((self.x_train, self.y_train))

        for e in zipped_x_y_train:
            label = int(e[self.feature_len])
            sample = np.asarray(e[:self.feature_len])
            encoded_sample = self.calc_perm_enc(sample)
            # Note: predict methods get sample as input, not encoded sample
            pred_label = self.predict_perm(sample)

            label_idx = label - 1
            pred_label_idx = pred_label - 1
            
            #adaptive single-pass training
            # adding to the correct label class
            dot_sample_class = np.dot(encoded_sample, self.classes_perm_encodings[label_idx])
            norm_sample_class_mult = np.linalg.norm(encoded_sample)*np.linalg.norm(self.classes_perm_encodings[label_idx])
            cos_sim = dot_sample_class/norm_sample_class_mult if norm_sample_class_mult != 0 else 0
            add_val = self.gamma*(1 - cos_sim)*encoded_sample
            self.classes_perm_encodings[label_idx] = np.add(self.classes_perm_encodings[label_idx], add_val)
            
            # subtracting from wrong class
            if label != pred_label:
                dot_sample_pred_class = np.dot(encoded_sample, self.classes_perm_encodings[pred_label_idx])
                norm_sample_pred_class = np.linalg.norm(encoded_sample)*np.linalg.norm(self.classes_perm_encodings[pred_label_idx])
                sub_cos_sim = dot_sample_pred_class/norm_sample_pred_class if norm_sample_pred_class != 0 else 0
                sub_val = self.gamma*(1 - sub_cos_sim)*encoded_sample
                self.classes_perm_encodings[pred_label_idx] = np.subtract(self.classes_perm_encodings[pred_label_idx], sub_val)
            
        return
    
    def FV_adaptive_assoc_classifier(self):
        # encoding each feature and then bundling their associations
        zipped_x_y_train = np.column_stack((self.x_train, self.y_train))

        for e in zipped_x_y_train:
            label = int(e[self.feature_len])
            sample = np.asarray(e[:self.feature_len])
            encoded_sample = self.calc_assoc_enc(sample)
            # Note: predict methods get sample as input, not encoded sample
            pred_label = self.predict_assoc(sample)

            label_idx = label - 1
            pred_label_idx = pred_label - 1
            
            #adaptive single-pass training
            # adding to the correct label class
            dot_sample_class = np.dot(encoded_sample, self.classes_assoc_encodings[label_idx])
            norm_sample_class_mult = np.linalg.norm(encoded_sample)*np.linalg.norm(self.classes_assoc_encodings[label_idx])
            cos_sim = dot_sample_class/norm_sample_class_mult if norm_sample_class_mult != 0 else 0
            add_val = self.gamma*(1 - cos_sim)*encoded_sample
            self.classes_assoc_encodings[label_idx] = np.add(self.classes_assoc_encodings[label_idx], add_val)
            
            # subtracting from wrong class
            if label != pred_label:
                dot_sample_pred_class = np.dot(encoded_sample, self.classes_assoc_encodings[pred_label_idx])
                norm_sample_pred_class = np.linalg.norm(encoded_sample)*np.linalg.norm(self.classes_assoc_encodings[pred_label_idx])
                sub_cos_sim = dot_sample_pred_class/norm_sample_pred_class if norm_sample_pred_class != 0 else 0
                sub_val = self.gamma*(1 - sub_cos_sim)*encoded_sample
                self.classes_assoc_encodings[pred_label_idx] = np.subtract(self.classes_assoc_encodings[pred_label_idx], sub_val)
            
        return
    
    def FV_adaptive_kernel_classifier(self):
        # encoding each feature and then bundling their kernel projection
        zipped_x_y_train = np.column_stack((self.x_train, self.y_train))

        for e in zipped_x_y_train:
            label = int(e[self.feature_len])
            sample = np.asarray(e[:self.feature_len])
            encoded_sample = self.calc_kernel_enc(sample)
            # Note: predict methods get sample as input, not encoded sample
            pred_label = self.predict_kernel(sample)

            label_idx = label - 1
            pred_label_idx = pred_label - 1
            
            #adaptive single-pass training
            # adding to the correct label class
            dot_sample_class = np.dot(encoded_sample, self.classes_kernel_encodings[label_idx])
            norm_sample_class_mult = np.linalg.norm(encoded_sample)*np.linalg.norm(self.classes_kernel_encodings[label_idx])
            cos_sim = dot_sample_class/norm_sample_class_mult if norm_sample_class_mult != 0 else 0
            add_val = self.gamma*(1 - cos_sim)*encoded_sample
            self.classes_kernel_encodings[label_idx] = np.add(self.classes_kernel_encodings[label_idx], add_val)
            
            # subtracting from wrong class
            if label != pred_label:
                dot_sample_pred_class = np.dot(encoded_sample, self.classes_kernel_encodings[pred_label_idx])
                norm_sample_pred_class = np.linalg.norm(encoded_sample)*np.linalg.norm(self.classes_kernel_encodings[pred_label_idx])
                sub_cos_sim = dot_sample_pred_class/norm_sample_pred_class if norm_sample_pred_class != 0 else 0
                sub_val = self.gamma*(1 - sub_cos_sim)*encoded_sample
                self.classes_kernel_encodings[pred_label_idx] = np.subtract(self.classes_kernel_encodings[pred_label_idx], sub_val)
            
        return
    
    def calc_assoc_enc(self, sample):
        ratio = 0
        encoded_sample = np.asarray([0 for i in range(self.dim)])

        for i, e in np.ndenumerate(sample):
            feature_idx = i[0]
            id_hv = self.id_assoc[feature_idx]
            feature_hv = self.assign_level_hv(e)
            feature_hv = np.multiply(feature_hv, id_hv)
            encoded_sample = np.add(encoded_sample, feature_hv)

        return encoded_sample

    def FV_assoc_classifier(self):
        # encoding each feature and then bundling their binding with identification vectors
        zipped_x_y_train = np.column_stack((self.x_train, self.y_train))

        for e in zipped_x_y_train:
            label = int(e[self.feature_len])
            sample = np.asarray(e[:self.feature_len])
            idx = label - 1
            self.classes_assoc_encodings[idx] = np.add(self.classes_assoc_encodings[idx], self.calc_assoc_enc(sample))
      
        return
    
    def predict_assoc(self, sample):
        assoc_encoded_sample = self.calc_assoc_enc(sample)
        predicted_label = self.cosine(assoc_encoded_sample, self.classes_assoc_encodings)
        return predicted_label
    
    def tanh_activation(self, x):
        t = (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
        return t
    
    
    def normalize_sample(self, sample):
        norm_sample = np.zeros(len(sample))
        for idx, e in np.ndenumerate(sample):
            norm_sample[idx] = (e - self.min_x_train)/(self.max_x_train - self.min_x_train)

        return norm_sample
    
    def calc_kernel_enc(self, sample):
        norm_sample = self.normalize_sample(sample)
        projected = []
        for row in self.kernel:
            dot_prod = np.dot(norm_sample, row)
            tanh_val = self.tanh_activation(dot_prod)
            binarized_val = 1 if tanh_val > 0 else -1
            projected.append(binarized_val)

        return np.asarray(projected)
    
    def FV_kernel_classifier(self):
        # encoding each feature and then bundling their binding with identification vectors
        zipped_x_y_train = np.column_stack((self.x_train, self.y_train))

        for e in zipped_x_y_train:
            label = int(e[self.feature_len])
            sample = np.asarray(e[:self.feature_len])
            idx = label - 1
            self.classes_kernel_encodings[idx] = np.add(self.classes_kernel_encodings[idx], self.calc_kernel_enc(sample))
            
        return

    def predict_kernel(self, sample):
        kernel_encoded_sample = self.calc_kernel_enc(sample)
        predicted_label = self.cosine(kernel_encoded_sample, self.classes_kernel_encodings)
        return predicted_label
    
    
    def retraining_perm(self):
        zipped_x_y_train = np.column_stack((self.x_train, self.y_train))
        
        for e in zipped_x_y_train:
            label = int(e[self.feature_len])
            sample = np.asarray(e[:self.feature_len])
            encoded_sample = self.calc_perm_enc(sample)
            predicted = self.predict_perm(sample)

            if predicted == label:
                continue
            else:
                predicted_idx = predicted - 1
                label_idx = label - 1
                self.classes_perm_encodings[predicted_idx] = np.subtract(self.classes_perm_encodings[predicted_idx], encoded_sample)
                self.classes_perm_encodings[label_idx] = np.add(self.classes_perm_encodings[label_idx], encoded_sample)

        return
    
    
    def retraining_assoc(self):
        zipped_x_y_train = np.column_stack((self.x_train, self.y_train))

        for e in zipped_x_y_train:
            label = int(e[self.feature_len])
            sample = np.asarray(e[:self.feature_len])
            encoded_sample = self.calc_assoc_enc(sample)
            predicted = self.predict_assoc(sample)

            if predicted == label:
                continue
            else:
                predicted_idx = predicted - 1
                label_idx = label - 1
                self.classes_assoc_encodings[predicted_idx] = np.subtract(self.classes_assoc_encodings[predicted_idx], encoded_sample)
                self.classes_assoc_encodings[label_idx] = np.add(self.classes_assoc_encodings[label_idx], encoded_sample)

        return
    
    
    def retraining_kernel(self):
        zipped_x_y_train = np.column_stack((self.x_train, self.y_train))

        for e in zipped_x_y_train:
            label = int(e[self.feature_len])
            sample = np.asarray(e[:self.feature_len])
            encoded_sample = self.calc_kernel_enc(sample)
            predicted = self.predict_kernel(sample)

            if predicted == label:
                continue
            else:
                predicted_idx = predicted - 1
                label_idx = label - 1
                self.classes_kernel_encodings[predicted_idx] = np.subtract(self.classes_kernel_encodings[predicted_idx], encoded_sample)
                self.classes_kernel_encodings[label_idx] = np.add(self.classes_kernel_encodings[label_idx], encoded_sample)
                
        return



In [336]:
def get_perm_acc(model, num):
    test_len = len(y_test[:num])
    counter = 0
    
    for i in range(len(x_test[:num])):
        pred = model.predict_perm(x_test[i])
        label = int(y_test[i])
        if pred == label:
            counter += 1
            
    return counter/test_len

def get_assoc_acc(model, num):
    test_len = len(y_test[:num])
    counter = 0
    for i in range(len(x_test[:num])):
        pred = model.predict_assoc(x_test[i])
        label = int(y_test[i])
        if pred == label:
            counter += 1
            
    return counter/test_len

def get_kernel_acc(model, num):
    test_len = len(y_test[:num])
    counter = 0
    for i in range(len(x_test[:num])):
        pred = model.predict_kernel(x_test[i])
        label = int(y_test[i])
        if pred == label:
            counter += 1
            
    return counter/test_len

### Part 1: Accuracy gained for Permutaion-based, Association-based, and Kernel-based encodings on ISOLET dataset

In [337]:
hdc = hd_classifier()

In [333]:
print('training perm-based classifier...')
hdc.FV_perm_classifier()
print('Done! the accuracy on test data for perm-based classifier is: ')
perm_acc = get_perm_acc(hdc, len(y_test))
print(perm_acc)

training perm-based classifier...
Done! the accuracy on test data for perm-based classifier is: 
0.7543296985246953


In [338]:
print('training assoc-based classifier...')
hdc.FV_assoc_classifier()
print('Done! the accuracy on test data for assoc-based classifier is: ')
assoc_acc = get_assoc_acc(hdc, len(y_test))
print(assoc_acc)

training assoc-based classifier...
Done! the accuracy on test data for assoc-based classifier is: 
0.8050032071840924


In [339]:
print('training kernel-based classifier...')
hdc.FV_kernel_classifier()
print('Done! the accuracy on test data for kernel-based classifier is: ')
kernel_acc = get_kernel_acc(hdc, len(y_test))
print(kernel_acc)

training kernel-based classifier...
Done! the accuracy on test data for kernel-based classifier is: 
0.7915330339961514


| Model   |      accuracy      | 
|----------|---------------|
| Permutation-based |  %75.43 |
| Associacion-based |    %80.50   |
| Kernel-based | %79.15 |

### Part 2: Accuracy gained for Permutaion-based, Association-based, and Kernel-based encodings on ISOLET dataset with applying retraining on each model 
##### (here we use 2 pass on the whole training data for each model retraining process)

In [340]:
retraining_perm_acc = []
for i in range(2):
    print('pass no. ' + str(i + 1) + 'retraining for perm-based encoding...')
    hdc.retraining_perm()
    print('Done! new accuracy on the permutaion-based model:')
    t = get_perm_acc(hdc, len(y_test))
    print(t)
    retraining_perm_acc.append(t)

pass no. 1retraining for perm-based encoding...
Done! new accuracy on the permutaion-based model:
0.7613855035279025
pass no. 2retraining for perm-based encoding...
Done! new accuracy on the permutaion-based model:
0.8088518280949326


In [341]:
retraining_assoc_acc = []
for i in range(2):
    print('pass no. ' + str(i + 1) + ' retraining for assoc-based encoding...')
    hdc.retraining_assoc()
    print('Done! new accuracy on the association-based model:')
    l = get_assoc_acc(hdc, len(y_test))
    print(l)
    retraining_assoc_acc.append(l)

pass no. 1 retraining for assoc-based encoding...
Done! new accuracy on the association-based model:
0.8383579217447081
pass no. 2 retraining for assoc-based encoding...
Done! new accuracy on the association-based model:
0.8774855676715844


In [342]:
retraining_kernel_acc = []
for i in range(2):
    print('pass no. ' + str(i + 1) + ' retraining for kernel-based encoding...')
    hdc.retraining_kernel()
    print('Done! new accuracy on the kernel-based model:')
    m = get_kernel_acc(hdc, len(y_test))
    print(m)
    retraining_kernel_acc.append(m)

pass no. 1 retraining for kernel-based encoding...
Done! new accuracy on the kernel-based model:
0.8678640153944837
pass no. 2 retraining for kernel-based encoding...
Done! new accuracy on the kernel-based model:
0.8717126363053239


| Model   |      1 pass retraining accuracy    | 2 pass retraining accuracy |
|----------|---------------|---------------|
| Permutation-based |  %76.13 |%80.88 |
| Associacion-based |    %83.83   |%87.74 |
| Kernel-based | %86.78 |%87.17 |

### Part 3: Accuracy gained for Permutaion-based, Association-based, and Kernel-based encodings on ISOLET dataset with applying adaptive single-pass training
##### (here we use gamma = 0.5 as learning rate)

In [349]:
adaptive_hdc = hd_classifier()

In [350]:
print('Adaptive single-pass perm-based classifier...')
adaptive_hdc.FV_adaptive_perm_classifier()
print('Done! the accuracy on test data for adaptive perm-based classifier is: ')
adaptive_perm_acc = get_perm_acc(adaptive_hdc, len(y_test))
print(adaptive_perm_acc)

Adaptive single-pass perm-based classifier...
Done! the accuracy on test data for adaptive perm-based classifier is: 
0.7613855035279025


In [351]:
print('Adaptive single-pass assoc-based classifier...')
adaptive_hdc.FV_adaptive_assoc_classifier()
print('Done! the accuracy on test data for adaptive assoc-based classifier is: ')
adaptive_assoc_acc = get_assoc_acc(adaptive_hdc, len(y_test))
print(adaptive_assoc_acc)

Adaptive single-pass assoc-based classifier...
Done! the accuracy on test data for adaptive assoc-based classifier is: 
0.7171263630532393


In [359]:
print('Adaptive single-pass kernel-based classifier...')
t = hd_classifier()
t.FV_adaptive_kernel_classifier()
print('Done! the accuracy on test data for adaptive kernel-based classifier is: ')
adaptive_kernel_acc = get_kernel_acc(t, len(y_test))
print(adaptive_kernel_acc)

Adaptive single-pass kernel-based classifier...
Done! the accuracy on test data for adaptive kernel-based classifier is: 
0.03848620910840282


#### Well, I checked the code couple of times but don't know what is happening with kernel-based adaptive single-pass training :/

### Part 4: Accuracy gained for Permutaion-based, Association-based, and Kernel-based encodings on ISOLET dataset with different dimentionalities
##### (since the retraining method takes about double time than the single pass one, and also the effect of dimentionality can be shown in both cases, we use single pass training here)

In [361]:
dim_test_perm = []
models = [hd_classifier((i+1)*1000) for i in range(5)]
for i in range(len(models)):
    model = models[i]
    model.FV_perm_classifier()
    acc = get_perm_acc(model, len(y_test))
    print('accuracy for dim= ' + str((i+1)*1000) + ' is:')
    print(acc)
    dim_test_perm.append(acc)

accuracy for dim= 1000 is:
0.21808851828094933
accuracy for dim= 2000 is:
0.7280307889672867
accuracy for dim= 3000 is:
0.6459268762026941
accuracy for dim= 4000 is:
0.7447081462475946
accuracy for dim= 5000 is:
0.7491982039769083


#### As we can see, the accuracy enhances with increasing the dimentionality
| Dim   | accuracy |
|----------|---------------|
| 1000 | %21.81 |
| 2000 | %72.80 |
| 3000 | %64.59 |
| 4000 | %74.47 |
| 5000 | %74.91 |

#### Part 5: The effect of batch size cannot be seen here since we are just accumulating the weights instead of adding them one by one. Actually time constraint is another reason this result is not available